# Creating and Backtesting Pivot Point Strategies

## Getting the (raw) Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
data = pd.read_csv("EURUSD_pivot.csv", index_col = "time", parse_dates = ["time"])
data

In [ ]:
data.info()

In [ ]:
data.index

In [ ]:
data.Close.plot(figsize = (12, 8), title = "EUR/USD", fontsize = 12)
plt.show()

## Preparing the Data (1) - Timezone Conversion

In [ ]:
data

In [ ]:
print(data.index.tz)

In [ ]:
data = data.tz_localize("UTC") # localize to UTC time
data

In [ ]:
data.index

In [ ]:
data = data.tz_convert("US/Eastern") # convert to US/Eastern (NY) time
data

In [ ]:
print(data.index.tz)

## Preparing the Data (2) - Resampling to daily (NY Close)

In [ ]:
close = data.Close.to_frame().copy()

In [ ]:
close.head(15)

In [ ]:
close.resample("D").last()

In [ ]:
close.head(60)

In [ ]:
close.resample("D", offset = "17H").last().dropna()

## Preparing the Data (3) - OHLC Resampling

In [ ]:
data

In [ ]:
agg_dict = {"Open":"first",
            "High":"max",
            "Low":"min",
            "Close":"last"
           }

In [ ]:
data.resample("D", offset = "17H").agg(agg_dict).dropna()

In [ ]:
daily_data = data.resample("D", offset = "17H").agg(agg_dict).dropna()
daily_data

In [ ]:
daily_data.info()

In [ ]:
data

## Preparing the Data (4) - Merging Intraday and Daily Data

In [ ]:
daily_data

In [ ]:
daily_data.columns = ["Open_d", "High_d", "Low_d", "Close_d"]

In [ ]:
daily_data

In [ ]:
data

In [ ]:
daily_data.shift().dropna()

In [ ]:
pd.concat([data, daily_data.shift().dropna()], axis = 1).ffill().dropna().head(60)

In [ ]:
data = pd.concat([data, daily_data.shift().dropna()], axis = 1).ffill().dropna()

In [ ]:
data

In [ ]:
data.info()

## Pivot Point - Overview and Data requirements

In [ ]:
data # already prepared

In [ ]:
data.index # New York time / US Eastern

- Currencies trade 24h / 5 days -> from Sunday 5pm (__US Eastern__) to Friday 5pm
- Close is at __5pm__ US Eastern

- Pivot Point is an __Intraday__ Price Action Strategy
- It uses (Open), High, Low and Close Prices of the __previous day__

In [ ]:
data.head(60)

## Adding Pivot Point and Support and Resistance Lines

In [ ]:
data

__Pivot Point Line__: The average of the previous day´s High, Low and Close price

In [ ]:
data["PP"] = (data.High_d + data.Low_d + data.Close_d) / 3
data

__S1 and S2 Support Lines__

In [ ]:
data["S1"] = data.PP * 2 - data.High_d

In [ ]:
data["S2"] = data.PP - (data.High_d - data.Low_d )

In [ ]:
data

__R1 and R2 Resistance Lines__

In [ ]:
data["R1"] = data.PP * 2 - data.Low_d

In [ ]:
data["R2"] = data.PP + (data.High_d - data.Low_d )

In [ ]:
data

In [ ]:
data.loc["2019-06-17":"2019-06-25", ["Open", "PP", "S1", "S2", "R1", "R2"]].plot(figsize = (12, 8))
plt.title("EUR/USD - Pivot Point", fontsize = 20)
plt.show()

## Defining a simple Pivot Point Strategy

In [ ]:
data.loc["2019-06-17":"2019-06-19", ["Open", "PP", "S1", "S2", "R1", "R2"]].plot(figsize = (12, 8))
plt.title("EUR/USD - Pivot Point", fontsize = 20)
plt.show()

__There is not the one Pivot Point Strategy that is set in stone__. <br>
Working with Price Action and Support and Resistance Lines is __highly subjective__ and __case specific__. <br>
Subject to __human__ intuition, judgement and experience -> Can be used for Algorithmic Trading?  

But, the consensus is that 
- a __price above PP__ signals __bullish__ sentiment -> go long
- a __price below PP__ signals __bearish__ sentiment -> go short

__Example: S1/R1 signal end of trend as probability of reversal increases (exit points)__

- If the price reaches S1 -> go neutral
- If the price reaches R1 -> go neutral

In [ ]:
data["position"] = np.where(data.Open > data.PP, 1, -1)

In [ ]:
data["position"] = np.where(data.Open >= data.R1, 0, data.position)

In [ ]:
data["position"] = np.where(data.Open <= data.S1, 0, data.position)

In [ ]:
data.position = data.position.fillna(0)

In [ ]:
data.position.value_counts()

## Vectorized Strategy Backtesting

In [ ]:
data

In [ ]:
data.Open.shift(-1)

In [ ]:
data["returns"] = np.log(data.Open.shift(-1).div(data.Open))

In [ ]:
data["strategy"] = data.position * data["returns"]

In [ ]:
data

In [ ]:
data.dropna(inplace = True)

In [ ]:
data

In [ ]:
ptc = 0.00007

In [ ]:
data["trades"] = data.position.diff().fillna(0).abs()

In [ ]:
data.trades.value_counts()

In [ ]:
data["strategy_net"] = data.strategy - data.trades * ptc

In [ ]:
data["creturns"] = data["returns"].cumsum().apply(np.exp)
data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
data["cstrategy_net"] = data["strategy_net"].cumsum().apply(np.exp)

In [ ]:
data

In [ ]:
data[["creturns", "cstrategy", "cstrategy_net"]].plot(figsize = (12 , 8))
plt.show()

In [ ]:
data.loc["2020", ["creturns", "cstrategy", "cstrategy_net"]].plot(figsize = (12 , 8))
plt.show()